## Notebook that will evaluate the KNN model

In [1]:
#importing libraires to use
import sklearn.preprocessing as skl_pre
import sklearn.neighbors as skl_nb
import sklearn.model_selection as skl_ms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import copy
from sklearn.metrics import confusion_matrix

#need to get the processing python file in another directory
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from process_data_for_test_set import process_data
from evaluate import evaluate_pred, print_dummy_evaluations
from visualization import plot_confusion_matrix

Below we get the entire training set and the real test set all unified with preprocessing

In [3]:
#getting the percentage splits of our training and testing data
# split_prec = {
#     'train': 0.8, 
#     'test': 0.2,
# }
#whatever scaler we will use
#scaler = skl_pre.StandardScaler()

#get the training and testing data
X_train, Y_train, real_test = process_data()
print(X_train)
# print(X_test)
print(Y_train)
# print(Y_test)
print(real_test)

Dropped columns: ['snow']
New columns: ['good_weather' 'is_day' 'temp_fahrenheit']
Dropped columns: ['snow']
New columns: ['good_weather' 'is_day' 'temp_fahrenheit']
Split: "train" 	[Size: 1600] 	[Prec: 1.0]
	X: (1600, 17)
	Y: (1600,)
      hour_of_day  day_of_week     month  holiday  weekday  summertime  \
0        0.608696     0.833333  0.818182      0.0      0.0         1.0   
1        0.000000     1.000000  0.363636      0.0      0.0         1.0   
2        0.565217     0.666667  0.363636      0.0      1.0         1.0   
3        1.000000     0.166667  0.545455      0.0      1.0         1.0   
4        0.391304     0.333333  0.454545      0.0      1.0         1.0   
...           ...          ...       ...      ...      ...         ...   
1595     1.000000     0.833333  0.636364      0.0      0.0         1.0   
1596     0.000000     0.833333  1.000000      0.0      0.0         0.0   
1597     0.304348     0.333333  0.727273      0.0      1.0         1.0   
1598     0.478261     0.5

In [11]:
X_train

,hour_of_day,day_of_week,month,holiday,weekday,summertime,temp,dew,humidity,precip,snowdepth,windspeed,cloudcover,visibility,temp_fahrenheit,good_weather,is_day
0,0.608696,0.833333,0.818182,0.0,0.0,1.0,0.762864,0.819672,0.522965,0.000000,0.0,0.497717,0.813,1.000000,0.7625,0.847791,1.0
1,0.000000,1.000000,0.363636,0.0,0.0,1.0,0.440716,0.639344,0.872085,0.010320,0.0,0.479452,0.974,0.666667,0.4375,0.653045,0.0
2,0.565217,0.666667,0.363636,0.0,1.0,1.0,0.769575,0.829040,0.523917,0.000000,0.0,0.566210,0.813,1.000000,0.7750,0.854297,1.0
3,1.000000,0.166667,0.545455,0.0,1.0,1.0,0.834452,0.971897,0.693241,0.000000,0.0,0.292237,0.244,1.000000,0.8375,0.952170,0.0
4,0.391304,0.333333,0.454545,0.0,1.0,1.0,0.633110,0.796253,0.756069,0.000000,0.0,0.602740,0.998,1.000000,0.6375,0.794349,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,1.000000,0.833333,0.636364,0.0,0.0,1.0,0.682327,0.861827,0.797120,0.000000,0.0,0.347032,0.813,1.000000,0.6875,0.846323,0.0
1596,0.000000,0.833333,1.000000,0.0,0.0,0.0,0.281879,0.426230,0.720609,0.000000,0.0,0.000000,0.000,1.000000,0.2750,0.318822,0.0
1597,0.304348,0.333333,0.727273,0.0,1.0,1.0,0.711409,0.927400,0.892075,0.000000,0.0,0.205479,0.589,1.000000,0.7125,0.892181,0.0
1598,0.478261,0.500000,0.181818,0.0,1.0,1.0,0.425056,0.637002,0.914802,0.002126,0.0,0.253425,1.000,0.981132,0.4250,0.649750,1.0


Below are the best hyperparameters found from the KNN folder

In [7]:
best_cols = ['hour_of_day',
  'day_of_week',
  'holiday',
  'weekday',
  'dew',
  'humidity',
  'snowdepth',
  'windspeed',
  'visibility',
  'temp_fahrenheit',
  'good_weather',
  'is_day']
best_params = {'n_neighbors': 11, 'p': 1, 'weights': 'uniform'}

So we will train the entire training set with these hyperparameters then use that model to make our final test set predictions

In [12]:
#training knn model with the best parameters
best_knn = skl_nb.KNeighborsClassifier(n_neighbors=best_params["n_neighbors"], p=best_params["p"], weights=best_params["weights"])
best_knn.fit(X_train[best_cols],np.array(Y_train.values.tolist()).reshape(-1,))

KNeighborsClassifier(n_neighbors=11, p=1)

In [13]:
best_knn.score(X_train[best_cols],Y_train)

0.900625

In [14]:
test_predictions = best_knn.predict(real_test[best_cols])

In [17]:
test_predictions

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,

In [22]:
import csv
with open("predictions.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(test_predictions)